Function for image preview

In [1]:
import tkinter
from PIL import ImageTk, Image

def image_preview (path, size = (0, 0)):
    root = tkinter.Tk ()#Start window
    root.title ("Preview")#Modify this to change the title
    rawimg = Image.open(path)#Open image path

    if size != (0, 0):#If a specific resolution has been requested
        rawimg = rawimg.resize (size)#Change the image resolution

    img = ImageTk.PhotoImage(rawimg)#Adapt the image to tkinter
    panel = tkinter.Label (root, image = img)#Add image to the window
    panel.pack ()#Build window
    root.mainloop ()#Run window

    root.quit()

Function for live recording

In [2]:
def live_recording(data):
    from pvrecorder import PvRecorder
    import wave, struct
    import speech_recognition as sr
    import time

    for index, device in enumerate(PvRecorder.get_available_devices()):
        print(f"[{index}] {device}")

    recorder= PvRecorder(device_index= -1, frame_length=51200)
    audio= []
    path= 'test_audio.wav'
    try:
        recorder.start()

        while True:
            frame= recorder.read()
            audio.extend(frame)
    except KeyboardInterrupt:
        recorder.stop()
        with wave.open(path, 'w') as f:
            f.setparams((1, 2, 16000, 51200, "NONE", "NONE"))
            f.writeframes(struct.pack("h" * len(audio), *audio))
    finally:
        recorder.delete()

    r = sr.Recognizer()
    with sr.AudioFile('test_audio.wav') as source:
        audio_text = r.listen(source)

    dialogue= r.recognize_google(audio_text, show_all=False) 


    subj= TextBlob(dialogue).subjectivity
    temp= analyzer.polarity_scores(dialogue)
    comp= temp['compound']

    new_row= {"Text": dialogue,"Compound": comp, "Subjectivity": subj,"Compound_T":0, "Subjectivity_T": 0,	"KMeans_5": 0 }
    data= data.append(new_row, ignore_index=True)

    data[['Compound_T', 'Subjectivity_T']]= scaler.fit_transform(data[['Compound', 'Subjectivity']])

    kmeans= KMeans(n_clusters=5)
    kmeans.fit(data[['Compound_T', "Subjectivity_T"]])
    data['KMeans_5']= kmeans.labels_

    return data



Function for the output

In [3]:
def eco_output(data):
    x= len(data.index) -1

    
    if(data.iloc[23588]['KMeans_5']== data.iloc[x]['KMeans_5']):
        print(data.iloc[x])
        image_preview("C:/Users/tysco/Downloads/red-smiley-face-hi.png")
    elif(data.iloc[23589]['KMeans_5']== data.iloc[x]['KMeans_5']):
        print(data.iloc[x])
        image_preview("C:/Users/tysco/Downloads/3267476.png")
    elif(data.iloc[23590]['KMeans_5']== data.iloc[x]['KMeans_5']):
        print(data.iloc[x])
        image_preview("C:/Users/tysco/Downloads/7876417.png")
    elif(data.iloc[23591]['KMeans_5']== data.iloc[x]['KMeans_5']):   
        print(data.iloc[x])
        image_preview("C:/Users/tysco/Downloads/download.png")
    #elif(data.iloc[23592]['KMeans_5']== data.iloc[x]['KMeans_5']):
    else:
        print(data.iloc[x])
        image_preview("C:/Users/tysco/Downloads/029645d6d5de5ffcd7596491a509f31e.jpg")       
    #else:
    #   print("error")

Function for text input

In [4]:
def type_input(data):

    dialogue= input()

    subj= TextBlob(dialogue).subjectivity
    temp= analyzer.polarity_scores(dialogue)
    comp= temp['compound']

    new_row= {"Text": dialogue,"Compound": comp, "Subjectivity": subj,"Compound_T":0, "Subjectivity_T": 0,	"KMeans_5": 0 }
    data= data.append(new_row, ignore_index=True)

    data[['Compound_T', 'Subjectivity_T']]= scaler.fit_transform(data[['Compound', 'Subjectivity']])

    kmeans= KMeans(n_clusters=5)
    kmeans.fit(data[['Compound_T', "Subjectivity_T"]])
    data['KMeans_5']= kmeans.labels_
    
    return data



function to read in from data file

In [ ]:
# reading the file 
#dfile = my_file.read() 
  
# replacing end of line('/n') with ' ' and 
# splitting the text it further when '.' is seen. 
#data_list=[]
#data_list = dfile.replace('\n', ' ').split(".") 
  
# printing the data 
#print(data_list) 
#my_file.close() 

#length= len(data_list)

#for i in range(length):
#    ex= data_list[i]
#    #print(ex)
#    subj= TextBlob(ex).subjectivity
#    temp= analyzer.polarity_scores(ex)
#    comp= temp['compound']
#    new_row= {"Text": ex,"Compound": comp, "Subjectivity": subj}
#    data= data.append(new_row, ignore_index=True)
#    '''

    

Main Function

In [5]:
#importing libraries to program
#evaluates sentiment and sunjectivity
from textblob import TextBlob   
#evaluates sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer    
#numbers manipulation lirbary
import numpy as np   
#dataframe library 
import pandas as pd 
#plots lirbrary
import matplotlib.pyplot as plt  
import matplotlib.image as mpimg
#stat data library 
import seaborn as sns   
#image library 
from PIL import Image 
import time

import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans


#importing data from csv file 
df= pd.read_csv('Excosound_data.csv') 

#data frame for building the program 
dataa= df

#print(df.shape)
#df= df.head(500)
#testframe= dataa
#print(df.shape)
#print(testframe.shape)



#setting analayzer for vader
analyzer= SentimentIntensityAnalyzer()
#setting dataframe to use during program
data= pd.DataFrame(columns= ['Text', 'Compound', 'Subjectivity'])
fulldata= pd.DataFrame(columns= ['Text', 'Compound', 'Subjectivity'])


#display datframe
data

#loop to run the program through the dataframe until there aren't any more rows
for index, row in dataa.iterrows():
    #display text 
    ex= row['Text']
    #analyzing sentiment with vader analyzer and subjectiity with TextBlob
    subj= TextBlob(ex).subjectivity
    temp= analyzer.polarity_scores(ex)
    comp= temp['compound']

    print(" ")
    #printing subjectivity
    print(subj)
    #printing sentiment 
    print(comp)

    data.loc[len(data.index)] = [ex, comp, subj]
    fulldata.loc[len(dataa.index)] = [ex, comp, subj]


data.dropna(inplace=True)   #removing any empty lines 

scaler= StandardScaler()  #setting scaler

data[['Compound_T', 'Subjectivity_T']]= scaler.fit_transform(data[['Compound', 'Subjectivity']]) #transforming data 

kmeans= KMeans(n_clusters=5)    #setting kmeans algorithm for 5 clusters 

kmeans.fit(data[['Compound_T', "Subjectivity_T"]])   #fitting data to the 5 clusters

data['KMeans_5']= kmeans.labels_    #adding data to dataframe 
    

 
0.6333333333333333
-0.9349
 
0.6333333333333333
-0.9349
 
0.6333333333333333
-0.9349
 
0.6333333333333333
-0.9349
 
0.6333333333333333
-0.9349
 
0.6333333333333333
-0.9349
 
0.6333333333333333
-0.9349
 
0.6333333333333333
-0.9349
 
0.6333333333333333
-0.9349
 
0.6333333333333333
-0.9349
 
0.6333333333333333
-0.9349
 
0.6333333333333333
-0.9349
 
0.6333333333333333
-0.9349
 
0.6333333333333333
-0.9349
 
0.6333333333333333
-0.9349
 
0.6333333333333333
-0.9349
 
0.6333333333333333
-0.9349
 
0.6333333333333333
-0.9349
 
0.6333333333333333
-0.9349
 
0.6333333333333333
-0.9349
 
0.6333333333333333
-0.9349
 
0.6333333333333333
-0.9349
 
0.6333333333333333
-0.9349
 
0.46388888888888885
-0.2263
 
0.46388888888888885
-0.2263
 
0.46388888888888885
-0.2263
 
0.46388888888888885
-0.2263
 
0.46388888888888885
-0.2263
 
0.46388888888888885
-0.2263
 
0.46388888888888885
-0.2263
 
0.46388888888888885
-0.2263
 
0.46388888888888885
-0.2263
 
0.46388888888888885
-0.2263
 
0.46388888888888885
-0.2263
 
0

c:\Users\tysco\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [6]:
data= live_recording(data)
eco_output(data)
SystemExit

[0] Microphone (2- USB Audio Device)
[1] Internal Microphone (Conexant ISST Audio)


C:\Users\tysco\AppData\Local\Temp\ipykernel_17992\2700200429.py:39: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data= data.append(new_row, ignore_index=True)
c:\Users\tysco\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Text              everything I love about life is gone things ha...
Compound                                                     0.1082
Subjectivity                                               0.453333
Compound_T                                                 0.107088
Subjectivity_T                                            -0.163767
KMeans_5                                                          3
Name: 23593, dtype: object


SystemExit

K CLusters

In [ ]:
data.to_csv('Excosound_data.csv', )